In [5]:
# pip install librosa

In [7]:
#pip install opencv-python

In [1]:
# #This is for STFT

# import os
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import librosa
# import librosa.display
# from PIL import Image

# # Set the input and output directories
# gen_dir = "/Users/xaichotic/Library/CloudStorage/OneDrive-NorthumbriaUniversity-ProductionAzureAD/Term3/"

# input_folder = os.path.join(gen_dir, "Data_ALL/ECG/ECG_csv_files_labeled_baseline6_pantompkins")
# output_folder = os.path.join(gen_dir, "Data_ALL/ECG_images_baseline6_STFT")

# # Create output directory if it does not exist
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# # Define the parameters for the STFT
# n_fft = 1024
# hop_length = 512

# # Loop through all the CSV files in the folder
# for filename in os.listdir(input_folder):
#     if filename.endswith(".csv"):
#         # Load the CSV file as a NumPy array
#         file_path = os.path.join(input_folder, filename)
#         ecg_signal = pd.read_csv(file_path, header=None).values.flatten()

#         # Compute the STFT
#         stft = librosa.stft(ecg_signal, n_fft=n_fft, hop_length=hop_length)
#         stft_mag_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)

#         # Normalize the STFT to [0, 1] range
#         stft_norm = (stft_mag_db - np.min(stft_mag_db)) / np.ptp(stft_mag_db)

#         # Convert the STFT to a 8-bit grayscale image
#         stft_img = (stft_norm * 255).astype(np.uint8)

#         # Save the image as a PNG file
#         output_filename = os.path.splitext(filename)[0] + ".png"
#         output_path = os.path.join(output_folder, output_filename)
#         Image.fromarray(stft_img, mode="L").save(output_path)

#         print(f"Processed {filename} and saved {output_filename}")


In [1]:
import tensorflow as tf
print(tf.__version__)

2.12.0


In [2]:
#counting the length of the files EDA
#62, it matches the number of files in csv

gen_dir = "/Users/w20009829/OneDrive - Northumbria University - Production Azure AD/Term3/"

# Set the input and output directories
#gen_dir = "/Users/xaichotic/Library/CloudStorage/OneDrive-NorthumbriaUniversity-ProductionAzureAD/Term3/"

import os
folder_path = os.path.join(gen_dir, "Data_ALL/ECG_images_baseline6_STFT")

# Count the number of files in the folder
num_files = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])

print(f"There are {num_files} files in the {folder_path} folder.")


There are 81 files in the /Users/w20009829/OneDrive - Northumbria University - Production Azure AD/Term3/Data_ALL/ECG_images_baseline6_STFT folder.


In [2]:
#checking for the frequency if each class

import os

data_dir = os.path.join(gen_dir, "Data_ALL/ECG_images_baseline6_STFT")

class_counts = {0: 0, 1: 0}

for file_name in os.listdir(data_dir):
    if file_name.endswith(".png"):
        class_label = int(file_name.split("_")[-1].split(".")[0])
        class_counts[class_label] += 1

print(f"Class 0: {class_counts[0]} files")
print(f"Class 1: {class_counts[1]} files")

Class 0: 56 files
Class 1: 25 files


In [3]:
#extracting label from the filename

import cv2
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Set the data directory path
data_dir = os.path.join(gen_dir, "Data_ALL/ECG_images_baseline6_STFT")

X = []
y = []

# Go through all the files in the directory
for file_name in os.listdir(data_dir):
    if file_name.endswith(".png"):
        # Extract the label from the file name
        label = int(file_name.split("_")[-1][0])
        # Load the image file and preprocess it
        file_path = os.path.join(data_dir, file_name)
        img_gray = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        img_gray = cv2.resize(img_gray, (224, 224))
        img_rgb = cv2.cvtColor(img_gray, cv2.COLOR_GRAY2RGB) # Convert grayscale to RGB
        img_rgb = np.array(img_rgb).reshape(-1)
        # Save the preprocessed image and label in a list
        X.append(img_rgb)
        y.append(label)

# Convert the data to numpy arrays
X = np.array(X)
y = np.array(y)


In [4]:
from imblearn.under_sampling import RandomUnderSampler

# Apply random under-sampling to the data
rus = RandomUnderSampler()
X_resampled, y_resampled = rus.fit_resample(X, y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [5]:
# Print the shape and class counts of the resampled data
print("Resampled data shape:", X_resampled.shape)
unique, counts = np.unique(y_resampled, return_counts=True)
print("Class counts:", dict(zip(unique, counts)))

Resampled data shape: (50, 150528)
Class counts: {0: 25, 1: 25}


In [6]:
# Count the number of instances for each class in the resampled training data
class_counts = np.bincount(y_resampled)
print("Class distribution after Random Undersampling:", class_counts)

Class distribution after Random Undersampling: [25 25]


In [7]:
# Reshape the data into a 4D tensor
X_train = X_train.reshape((-1, 224, 224, 3))
X_test = X_test.reshape((-1, 224, 224, 3))

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(40, 224, 224, 3)
(40,)
(10, 224, 224, 3)
(10,)


In [11]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

import tensorflow as tf
import random

# Set the random seeds for reproducibility
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

batch_size = 32
neurons = 256

# Create ResNet50 model
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of ResNet50 model
for layer in resnet_model.layers:
    layer.trainable = False

# Add custom layers on top of ResNet50 model
model = Sequential()
model.add(resnet_model)
model.add(Flatten())
model.add(Dense(neurons, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
#opt = SGD(learning_rate=0.001)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Set up early stopping
#early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
early_stopping = EarlyStopping(monitor='val_loss', restore_best_weights=True, verbose=1, patience=10)

# Train the model with early stopping
history = model.fit(X_train, y_train, 
                    epochs=25, 
                    batch_size=batch_size, 
                    validation_data=(X_test, y_test), 
                    callbacks=[early_stopping])

test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test loss:", test_loss)
print("Test accuracy:", test_acc)


Epoch 1/25
2/2 [==============================] - 5s 2s/step - loss: 30.6582 - accuracy: 0.5250 - val_loss: 65.4861 - val_accuracy: 0.3000
Epoch 2/25
2/2 [==============================] - 2s 963ms/step - loss: 54.5902 - accuracy: 0.5000 - val_loss: 10.8840 - val_accuracy: 0.7000
Epoch 3/25
2/2 [==============================] - 2s 939ms/step - loss: 34.1017 - accuracy: 0.5250 - val_loss: 2.1447 - val_accuracy: 0.8000
Epoch 4/25
2/2 [==============================] - 2s 840ms/step - loss: 6.8443 - accuracy: 0.6750 - val_loss: 38.7712 - val_accuracy: 0.3000
Epoch 5/25
2/2 [==============================] - 2s 856ms/step - loss: 8.6473 - accuracy: 0.6500 - val_loss: 5.6799 - val_accuracy: 0.7000
Epoch 6/25
2/2 [==============================] - 2s 880ms/step - loss: 6.1091 - accuracy: 0.7000 - val_loss: 7.0022 - val_accuracy: 0.4000
Epoch 7/25
2/2 [==============================] - 2s 850ms/step - loss: 7.9637 - accuracy: 0.7000 - val_loss: 4.4908 - val_accuracy: 0.7000
Epoch 8/25
2/2 [=

1/1 [==============================] - 1s 1s/step
[[3 0]
 [1 6]]
              precision    recall  f1-score   support

           0       0.75      1.00      0.86         3
           1       1.00      0.86      0.92         7

    accuracy                           0.90        10
   macro avg       0.88      0.93      0.89        10
weighted avg       0.93      0.90      0.90        10

ROC AUC: 0.9523809523809524


In [12]:
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

# Get the predictions
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)

# Print the confusion matrix
cm = confusion_matrix(y_test, y_pred_classes)
print(cm)

# Print the classification report
print(classification_report(y_test, y_pred_classes))

# Print the ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred)
print('ROC AUC:', roc_auc)

import matplotlib.pyplot as plt

# Get the training and validation loss and accuracy values from the history object
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Create the figure and axis objects
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Plot the training and validation loss
ax1.plot(train_loss, label='Training Loss')
ax1.plot(val_loss, label='Validation Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()

# Plot the training and validation accuracy
ax2.plot(train_acc, label='Training Accuracy')
ax2.plot(val_acc, label='Validation Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.legend()

# Show the plot
plt.show()

cross-validation

In [14]:
#model for basic CNN

import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv1D, BatchNormalization, MaxPooling1D, Dropout, Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

# Set the random seeds for reproducibility
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

batch_size = 32
num_folds = 10
n_epochs =20
neurons = 256

# Create ResNet50 model
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of ResNet50 model
for layer in resnet_model.layers:
    layer.trainable = False

# Add custom layers on top of ResNet50 model
model = Sequential()
model.add(resnet_model)
model.add(Flatten())
model.add(Dense(neurons, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Define the CNN model
def create_model():
    model = Sequential()
    model.add(resnet_model)
    model.add(Flatten())
    model.add(Dense(neurons, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

In [16]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv1D, BatchNormalization, MaxPooling1D, Dropout, Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

# Set the random seeds for reproducibility
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = 42)

#K-fold cross-validation lists
score_list = []
precision_list = []
recall_list = []
f1_score_list = []
confusion_matrix_list = []
specificity_list = []
ROC_AUC_list = []


for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train, y_train)):
    #splitting the data into train and validation sets for this fold
    X_train_fold, y_train_fold = X_train[train_idx], y_train[train_idx]
    X_val_fold, y_val_fold = X_train[val_idx], y_train[val_idx]
    
    #defining the early stopping callback
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)
    
    #training the model for this fold
    model = create_model()
    history = model.fit(X_train_fold, y_train_fold, epochs=n_epochs, batch_size=batch_size, 
                        validation_data=(X_val_fold, y_val_fold), callbacks=[early_stop], verbose=1)
    
    #predicting the classes for the validation set for this fold
    y_pred = model.predict(X_val_fold)
    y_pred_classes = (y_pred > 0.5).astype(int)
    
    #calculating precision, recall, specificity, F1 score, AUC, and confusion matrix for this fold
    precision_list.append(precision_score(y_val_fold, y_pred_classes))
    recall_list.append(recall_score(y_val_fold, y_pred_classes))
    f1_score_list.append(f1_score(y_val_fold, y_pred_classes))
    #tp, fp, fn, tn = confusion_matrix(y_val_fold, y_pred_classes).ravel()
    tn, fp, fn, tp = confusion_matrix(y_val_fold, y_pred_classes).ravel()
    specificity = tn / (tn + fp)
    specificity_list.append(specificity)
    ROC_AUC_list.append(roc_auc_score(y_val_fold, y_pred))
    confusion_matrix_list.append(confusion_matrix(y_val_fold, y_pred_classes))

    #evaluating the model on the test set for this fold
    #_, test_acc = model.evaluate(X_test, y_test)
    _, test_acc = model.evaluate(X_val_fold, y_val_fold)

    score_list.append(test_acc)
    
    #printing the metrics and confusion matrix for this fold
    print(f'Fold {fold + 1}: Accuracy = {test_acc:.4f}, \
      Precision = {precision_list[-1]:.4f}, \
      Recall = {recall_list[-1]:.4f}, \
      Specificity = {specificity_list[-1]:.4f}, \
      AUC = {ROC_AUC_list[-1]:.4f}, \
      F1 Score = {f1_score_list[-1]:.4f}')
    print(f'Confusion matrix:\n{confusion_matrix_list[-1]}')
    
#Print the average scores for all folds

print(f'Average Accuracy: {np.mean(score_list):.4f} +/- {np.std(score_list):.4f}')
print(f'Average Precision: {np.mean(precision_list):.4f} +/- {np.std(precision_list):.4f}')
print(f'Average Recall/Sensitivity: {np.mean(recall_list):.4f} +/- {np.std(recall_list):.4f}')
print(f'Average Specificity: {np.mean(specificity_list):.4f} +/- {np.std(specificity_list):.4f}')
print(f'Average F1-score: {np.mean(f1_score_list):.4f} +/- {np.std(f1_score_list):.4f}')
print(f'Average ROC_AUC score: {np.mean(ROC_AUC_list):.4f} +/- {np.std(ROC_AUC_list):.4f}')

#printing the overall confusion matrix

overall_cm = np.sum(confusion_matrix_list, axis=0)
print(f'Overall confusion matrix:\n{overall_cm}')

Epoch 1/20
2/2 [==============================] - 5s 1s/step - loss: 36.3434 - accuracy: 0.3889 - val_loss: 8.4029 - val_accuracy: 0.5000
Epoch 2/20
2/2 [==============================] - 2s 557ms/step - loss: 35.4127 - accuracy: 0.5000 - val_loss: 9.1178 - val_accuracy: 0.5000
Epoch 3/20
2/2 [==============================] - 2s 538ms/step - loss: 16.8719 - accuracy: 0.6667 - val_loss: 31.6164 - val_accuracy: 0.7500
Epoch 4/20
2/2 [==============================] - 2s 663ms/step - loss: 27.7425 - accuracy: 0.5833 - val_loss: 7.8812 - val_accuracy: 0.7500
Epoch 5/20
2/2 [==============================] - 2s 551ms/step - loss: 14.1483 - accuracy: 0.5833 - val_loss: 9.3271 - val_accuracy: 0.2500
Epoch 6/20
2/2 [==============================] - 2s 550ms/step - loss: 7.2200 - accuracy: 0.7778 - val_loss: 44.5616 - val_accuracy: 0.5000
Epoch 7/20
2/2 [==============================] - 2s 649ms/step - loss: 12.4890 - accuracy: 0.7222 - val_loss: 36.8386 - val_accuracy: 0.5000
Epoch 7: early

C:\Anaconda3\envs\xyrus\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 209ms/step - loss: 7.8812 - accuracy: 0.7500
Fold 1: Accuracy = 0.7500,       Precision = 0.0000,       Recall = 0.0000,       Specificity = 1.0000,       AUC = 0.0000,       F1 Score = 0.0000
Confusion matrix:
[[3 0]
 [1 0]]
Epoch 1/20
2/2 [==============================] - 5s 2s/step - loss: 21.2131 - accuracy: 0.5278 - val_loss: 16.6009 - val_accuracy: 0.7500
Epoch 2/20
2/2 [==============================] - 2s 686ms/step - loss: 58.5464 - accuracy: 0.5278 - val_loss: 2.9998 - val_accuracy: 0.7500
Epoch 3/20
2/2 [==============================] - 2s 684ms/step - loss: 28.1931 - accuracy: 0.5833 - val_loss: 1.5708 - val_accuracy: 0.7500
Epoch 4/20
2/2 [==============================] - 2s 566ms/step - loss: 12.0146 - accuracy: 0.6389 - val_loss: 4.5430 - val_accuracy: 0.7500
Epoch 5/20
2/2 [==============================] - 2s 576ms/step - loss: 31.5974 - accuracy: 0.6389 - val_loss: 49.6356 - val_accuracy: 0.7500
Epoch 6/20
2/2 [============

C:\Anaconda3\envs\xyrus\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1/1 [==============================] - 0s 215ms/step - loss: 1.5708 - accuracy: 0.7500
Fold 2: Accuracy = 0.7500,       Precision = 0.0000,       Recall = 0.0000,       Specificity = 1.0000,       AUC = 1.0000,       F1 Score = 0.0000
Confusion matrix:
[[3 0]
 [1 0]]
Epoch 1/20
2/2 [==============================] - 5s 2s/step - loss: 34.4694 - accuracy: 0.3611 - val_loss: 71.7723 - val_accuracy: 0.5000
Epoch 2/20
2/2 [==============================] - 2s 682ms/step - loss: 59.8963 - accuracy: 0.5556 - val_loss: 11.4154 - val_accuracy: 0.5000
Epoch 3/20
2/2 [==============================] - 2s 687ms/step - loss: 13.1243 - accuracy: 0.5278 - val_loss: 1.2727 - val_accuracy: 0.7500
Epoch 4/20
2/2 [==============================] - 2s 562ms/step - loss: 12.7526 - accuracy: 0.6111 - val_loss: 2.0382 - val_accuracy: 0.7500
Epoch 5/20
2/2 [==============================] - 2s 698ms/step - loss: 11.8479 - accuracy: 0.5278 - val_loss: 0.0352 - val_accuracy: 1.0000
Epoch 6/20
2/2 [============

Epoch 3/20
2/2 [==============================] - 2s 579ms/step - loss: 11.1902 - accuracy: 0.6944 - val_loss: 61.3157 - val_accuracy: 0.5000
Epoch 4/20
2/2 [==============================] - 2s 696ms/step - loss: 41.4751 - accuracy: 0.5833 - val_loss: 18.2140 - val_accuracy: 0.2500
Epoch 5/20
2/2 [==============================] - 2s 701ms/step - loss: 8.4891 - accuracy: 0.7222 - val_loss: 15.3602 - val_accuracy: 0.2500
Epoch 6/20
2/2 [==============================] - 2s 568ms/step - loss: 13.7170 - accuracy: 0.6667 - val_loss: 29.8632 - val_accuracy: 0.5000
Epoch 7/20
2/2 [==============================] - 2s 593ms/step - loss: 14.2384 - accuracy: 0.6389 - val_loss: 37.0193 - val_accuracy: 0.5000
Epoch 8/20
2/2 [==============================] - 2s 635ms/step - loss: 5.5501 - accuracy: 0.8333 - val_loss: 39.9548 - val_accuracy: 0.5000
Epoch 8: early stopping
1/1 [==============================] - 0s 199ms/step - loss: 15.3602 - accuracy: 0.2500
Fold 8: Accuracy = 0.2500,       Preci